In [ ]:
# import everything and define a test runner function
from importlib import reload
from time import sleep

import bloomfilter, merkleblock

from block import Block
from bloomfilter import (
    BloomFilter,
    BIP37_CONSTANT,
)
from ecc import PrivateKey
from helper import (
    bit_field_to_bytes,
    decode_base58,
    hash160,
    hash256,
    little_endian_to_int,
    murmur3,
    run,
    SIGHASH_ALL,
)
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    SimpleNode,
    FILTERED_BLOCK_DATA_TYPE,
    TX_DATA_TYPE,
)
from script import p2pkh_script
from tx import (
    Tx,
    TxIn,
    TxOut,
)

In [ ]:
# Example Bloom Filter

bit_field_size = 10
bit_field = [0] * bit_field_size

h256 = hash256(b'hello world')
bit = int.from_bytes(h256, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

In [ ]:
# Example Bloom Filter 2

bit_field_size = 10
bit_field = [0] * bit_field_size

h = hash256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
h = hash256(b'goodbye')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

In [ ]:
# Example Bloom Filter 3

bit_field_size = 10
bit_field = [0] * bit_field_size

phrase1 = b'hello world'
h1 = hash256(phrase1)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase1)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
phrase2 = b'goodbye'
h1 = hash256(phrase2)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase2)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
print(bit_field)

In [ ]:
# Example BIP0037 Bloom Filter

field_size = 2
num_functions = 2
tweak = 42

bit_field_size = field_size * 8
bit_field = [0] * bit_field_size

for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

### Exercise 1

#### 1.1 Given a Bloom Filter with these parameters: size=10, function count=5, tweak=99, which bits are set after adding these items? 

 * `b'Hello World'`
 * `b'Goodbye!'`

#### 1.2. Make [this test](/edit/session8/bloomfilter.py) pass.

In [ ]:
# Exercise 1.1

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')

# bit_field_size is 8 * field_size
# create a bit field with the appropriate size

# for each item you want to add to the filter
    # iterate function_count number of times
        # BIP0037 spec seed is i*BIP37_CONSTANT + tweak
        # get the murmur3 hash given that seed
        # set the bit to be h mod the bit_field_size
        # set the bit_field at the index bit to be 1
# print the bit field converted to bytes using bit_field_to_bytes in hex

In [ ]:
# Exercise 1.2

reload(bloomfilter)
run(bloomfilter.BloomFilterTest('test_add'))

### Exercise 2

#### 2.1. Make [this test](/edit/session8/bloomfilter.py) pass.

#### 2.2. Do the following:

* Connect to a testnet node
* Load a filter for your testnet address
* Send a request for transactions from the block which had your previous testnet transaction
* Receive the merkleblock and tx messages.

In [ ]:
# Exercise 2.1

reload(bloomfilter)
run(bloomfilter.BloomFilterTest('test_filterload'))

In [ ]:
# Exercise 2.2

from bloomfilter import BloomFilter

block_hash = bytes.fromhex('<fill this in>')
passphrase = b'<fill this in>'  # FILL THIS IN
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
filter_size = 30
filter_num_functions = 5
filter_tweak = -1  # FILL THIS IN

# get the hash160 of the address using decode_base58
# create a bloom filter using the filter_size, filter_num_functions and filter_tweak above
# add the h160 to the bloom filter

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
# complete the handshake
# send the filterload command with bf.filterload() as the payload
# create a getdata message
# add_data (FILTERED_BLOCK_DATA_TYPE, block_hash) to request the block
# send the getdata message
# wait for the merkleblock command
# wait for the tx command
# print the envelope payload in hex

### Exercise 3

#### 3.1. Make [this test](/edit/session8/merkleblock.py) pass.

In [ ]:
# Exercise 3.1

reload(merkleblock)
run(merkleblock.MerkleBlockTest('test_is_valid'))

### Exercise 4

#### 4.1. You have been sent some unknown amount of testnet bitcoins to your address. 

Send all of it back (minus fees) to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv` using only the networking protocol.

In [ ]:
# Exercise 4.1

from merkleblock import MerkleBlock, MerkleTree

last_block_hex = '<fill this in>'
last_block = bytes.fromhex(last_block_hex)
passphrase = b'<fill this in>'  # FILL THIS IN
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
h160 = decode_base58(addr)
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
filter_size = 30
filter_num_functions = 5
filter_tweak = -1  # FILL THIS IN
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
# create a bloom filter using variables above
# add the h160 to the bloom filter
# complete the handshake
# send the 'filterload' command with bf.filterload() as the payload

# create GetHeadersMessage with the last_block as the start_block
# send a getheaders message

# wait for the headers message
# get the stream from the headers
# parse the headers message

# initialize the GetDataMessage
# loop through the blocks in the headers message
    # check that the proof of work on the block is valid
    # check that this block's prev_block is the last block
    # set the last block to the current hash
    # add_data(FILTERED_BLOCK_DATA_TYPE, last_block) to get_data_message
# send the get_data_message

# initialize prev_tx to None
# while prev_tx is None 
    # wait for the merkleblock or tx commands
    # initialize the stream from the envelope
    # if we have the merkleblock command
        # parse the MerkleBlock
        # check that the MerkleBlock is valid
    # else we have the tx command
        # parse the tx (prev)
        # loop through the enumerated tx outs (enumerate(prev.tx_outs))
            # if our output has the same address as our address (addr) we found it
                # we found our utxo. set prev_tx, prev_index, prev_amount
                # break
# create tx_in
# calculate the output amount (prev_amount - fee)
# create tx_out
# create transaction on testnet
# sign the one input we have
# serialize and hex to see what it looks like
# send this signed transaction on the network
# wait a sec so this message goes through to the other node sleep(1) 
# now ask for this transaction from the other node
# create a GetDataMessage
# add_data (TX_DATA_TYPE, tx_obj.hash()) to get data message
# send the GetDataMessage
# now wait for a response
# if we have the tx command
    # parse the tx
    # check that the ids are the same
        # yes! we got to what we wanted